In [1]:

from scipy.integrate import ode

%pylab inline

import numpy as np
from scipy import optimize

import matplotlib.pyplot as plt
matplotlib.use("Agg")
import scipy.io as sio
import scipy.signal
from pylab import *

asin=np.arcsin
sqrt=np.sqrt
sin=np.sin
cos=np.cos
pi=np.pi
exp=np.exp
tan=np.tan
I=1j
inf=np.inf

hilbert=scipy.signal.hilbert

54.75

In [ ]:
### Here are important constants

# Relative sizes of the high and low lattitude boxes

HLSize = .15
LLSize = .85

SD = 500.0*LLSize   # Surface layer depth in meters
DD = 3230.0  # Deep layer depth in meters

HD = 500*HLSize


K = 3.0 # Mixing coefficient in meters/year
KH = 10.0 # Mixing coefficient between high and deep boxes

SR = .95 # Fraction of TPP regenerated above 500m
SF = .002 # Fraction of TPP permanently incorporated into sediments
DR = .048 # Fraction of TPP regenerated below 500m
DN = .015 # Fraction of TPP of N that is regenerated via denitrification to N2
RP = 2.0e-4 # River input of dissolved P in mol P/m^2/year
RN = 6.0e-3 # River input of dissolved N in mol N/m^2/year
AN = 7.5e-3 # Atmospheric input of N in molN/m^2/year
RPhyto = 25.0 # N:P ratio in organic matter from phytoplankton
RPHigh = 10.0 # N:P ratio in high lattitude phytoplankton

RNFixers = 50.0 # N:P ratio in Nitrogen Fixers
muPhyto = 120.0 # Growth rate of high lattitude phytoplankton
muNF = 90.6 # Maximum growth rate of nitrogen fixing bacteria in units of 1/year
muPhyto = 91.25 # Maximum growth rate of phytoplankton in units of 1/year
PH = 3.0e-5 # Half saturation constant for Phosphate in mol P /m^3
NH = 5.0e-4   # Half saturation constant for Nitrate in mol N /m^3

PHHigh = 3.0e-4
NHHigh = 5.0e-3

M = 73.0    # Mortality in 1/year
MH = 110.0  # Hihgh Lattitude Mortality

### Next are the functions that define the right hand side of the system of equations

# We first define the dynamical variables:

# NFixer stands for the population of nitrogen fixers
# PhytoP stands to phytoplankton
# PS stands for the surface phosphate
# NS stands for surface nitrate
# ND stands for deep nitrate
# PD stands for deep phosphate


###

def dNFixer(NFixer,PS):
    return muNF*PS/(PS+PH)*NFixer - M*NFixer

def dPhytoP(PhytoP,PS,NS):
    return muPhyto*min([PS/(PS+PH),NS/(NS+NH)])*PhytoP - M*PhytoP

def dPS(PS,NS,PD,ND,PhytoP,NFixer):
    return -muNF*PS/(PS+PH)*NFixer/RNFixers+M*(SR)*NFixer/RNFixers+M*(SR)*PhytoP/RPhyto-muPhyto*min([PS/(PS+PH),NS/(NS+NH)])*PhytoP/RPhyto+K*(PD-PS)/SD+RP*LLSize/SD  

def dPSH(PSH,NSH,PD,ND,PhytoPH):
    return MH*SR*PhytoPH/RPHigh-muHL*min([PSH/(PSH+PHHigh),NSH/(NSH+NHHigh)])*PhytoPH/RPHigh+KH*(PD-PSH)/HD+RP*HLSize/HD  



def dNS(PS,NS,PD,ND,PhytoP,NFixer):
    return -muPhyto*min([PS/(PS+PH),NS/(NS+NH)])*PhytoP + M*(SR-0.75*DN)*NFixer + M*(SR-0.75*DN)*PhytoP +K*(ND-NS)/SD+(RN+AN)*LLSize/SD

def dNSH(PSH,NSH,PD,ND,PhytoPH):
    return -muHL*min([PSH/(PSH+PHHigh),NSH/(NSH+NHHigh)])*PhytoPH+MH*SR*PhytoPH+MH*(SR-0.75*DN)*PhytoPH+KH*(ND-NSH)/HD+(RN+AN)*HLSize/HD                                   
    

def dPD(PS,PSH,NS,NSH,PD,ND,PhytoP,PhytoPH,NFixer):
    return M*DR*NFixer/RNFixers*SD/DD+M*DR*PhytoP/RPhyto*SD/DD+MH*DR*PhytoPH/RPHigh*HD/DD - K*(PD-PS)/DD- KH*(PD-PSH)/DD  

def dND(PS,PSH,NS,NSH,PD,ND,PhytoP,PhytoPH,NFixer):
    return M*(DR-0.25*DN)*NFixer*SD/DD+M*(DR-0.25*DN)*PhytoP*SD/DD-K*(ND-NS)/DD-KH*(ND-NSH)/DD+MH*(DR-0.25*DN)*PhytoPH*HD/DD


def dPhytoPHigh(PhytoPH,PSH,NSH):
    return muHL*min([PSH/(PSH+PHHigh),NSH/(NSH+NHHigh)])*PhytoPH - MH*PhytoPH



def dState(t,state):
    PS=state[0]
    PSH=state[1]
    NS=state[2]
    NSH=state[3]
    PD=state[4]
    ND=state[5]
    PhytoP=state[6]
    PhytoPH = state[7]
    NFixer=state[8]
    
    dS = np.zeros(9)
    dS[0] = dPS(PS,NS,PD,ND,PhytoP,NFixer)
    dS[1] = dPSH(PSH,NSH,PD,ND,PhytoPH)
    dS[2] = dNS(PS,NS,PD,ND,PhytoP,NFixer)
    dS[3] = dNSH(PSH,NSH,PD,ND,PhytoPH)
    dS[4] = dPD(PS,PSH,NS,NSH,PD,ND,PhytoP,PhytoPH,NFixer)
    dS[5] = dND(PS,PSH,NS,NSH,PD,ND,PhytoP,PhytoPH,NFixer)
    dS[6] = dPhytoP(PhytoP,PS,NS)
    dS[7] = dPhytoPHigh(PhytoPH,PSH,NSH)
    dS[8] = dNFixer(NFixer,PS)
    return dS

###








